# Data Treatment (Poliqueta Database)

In this notebook I'm doing all the data treatments and adjustments necessary to produce high quality visualizations. 

This notebook has an output <font color='blue'>treated_db.csv</font> with a subset of the original database properly treated.

-----

Specifically, for the `poliqueta` database, the files we'll read are <font color='blue'>IBUFRJ27.07.2020 - visualização.xlsx</font> and <font color='blue'>MNRJP27.07.2020 - visualização.xls</font>.

<font color='red'>**p.s.:** the idea is to transform all the code in this notebook into a `.py` file with a CLI interface to parse a file and produce a treated csv file right away. </font>

is it feasible? as we're selecting plenty different columns... maybe selecting them from a .txt file might be a good solution

-----

<br>

# IBUFRJ27.07.2020 - visualização.xlsx

## Imports

In [1]:
import datetime
import unidecode
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualizations for data analytics
import seaborn as sns
import matplotlib.pyplot as plt

# proprietary functions in ./src/MNViz.py
from src.MNViz import *

## Importing data

In [2]:
excel = pd.ExcelFile('./data/IBUFRJ27.07.2020 - visualização.xlsx')
sheet_name = excel.sheet_names

print('The excel file contains the following sheets:', sheet_name)
print('\nDatabase is in sheet:', sheet_name[0])

The excel file contains the following sheets: ['Planilha1', 'Planilha2']

Database is in sheet: Planilha1


In [3]:
db = excel.parse(sheet_name[0], sep=';', encoding='utf-8-sig')
poliqueta = db.copy()

print(f'The database has {db.shape[0]} rows and {db.shape[1]} columns.')

The database has 4231 rows and 91 columns.


## Adjusting column names

### removing '\n', '\t', and other special characters

In [4]:
poliqueta.columns = [str(col).replace(r'\n','') for col in poliqueta.columns]

## Adjusting Determiners and Collectors Names

<font color='red' size='5'>**p.s.:** Determiner Last Name 1 contains the FULL name of the Researcher </font>

Determiner Last Name 2 is all empty.

In [5]:
names_col = ['Collector Last Name 1', 'Collector First Name 1', 
             'Collector Last Name 2', 'Collector First Name 2', 'Collector Last Name 3', 
             'Collector First Name 3', 'Collector Last Name 4', 'Collector First Name 4']

In [6]:
for name_col in names_col:
    if 'last' in name_col.lower():
        poliqueta[name_col] = poliqueta[name_col].apply(lambda x: treat_names(x, pos='full'))
    else:
        poliqueta[name_col] = poliqueta[name_col].apply(treat_names)

In [7]:
def getFirstName(name):
    name = str(name).split(';')
    if len(name) > 1:
        return name[0].strip()
    elif name[0] == 'nan':
        return np.NAN
    else:
        return name[0].strip()

In [10]:
# there's more than one determiner in thar column, so I'm getting only the first one
poliqueta['Determiner Last Name 1'] = poliqueta['Determiner Last Name 1'].apply(getFirstName)

### creating column joining First and Last names

I'm doing this only for the first collectors and determiners

In [11]:
# repteis['DeterminatorFirst_and_LastName'] = repteis['DeterminatorFirstName1'] + ' ' + repteis['DeterminatorLastName1']

poliqueta['CollectorFirst_and_LastName'] = poliqueta['Collector First Name 1'] + ' ' + poliqueta['Collector Last Name 1']
poliqueta['CollectorFirst_and_LastName2'] = poliqueta['Collector First Name 2'] + ' ' + poliqueta['Collector Last Name 2']

## Treating taxon columns

In [12]:
taxon_columns = ['Kingdom', 'Phylum 1', 'Class 1', 'Order 1', 'Family 1', 'Genus 1',
                 'Species 1']  # selecting taxonomy columns

treat_taxon_columns(poliqueta, taxon_columns)

Adjusting column Kingdom
Adjusting column Phylum 1
Adjusting column Class 1
Adjusting column Order 1
Adjusting column Family 1
Adjusting column Genus 1
Adjusting column Species 1


## Adjusting Genus and Species

In [13]:
# dica da Manoela: epiteto especifico deve ser todo minusculo (especie e subespecie, nesse caso)
poliqueta['Species 1'] = poliqueta['Species 1'].str.lower()

# genero é escrito com primeira letra maiúscula
poliqueta['Genus 1'] = poliqueta['Genus 1'].str.lower().str.capitalize()

<br>

## adding `Genero` and `Especie`together (they completely identify each animal's species)

In [14]:
poliqueta['genus_and_species'] = poliqueta['Genus 1'] + ' ' + poliqueta['Species 1']

poliqueta['genus_and_species'] = poliqueta['genus_and_species'].str.lower().str.capitalize()

## Catching Month and Year

date_columns= ['Cataloged Date','Return Date','Determined date 1', 'Start Date]

format: YYYY-mm-dd

<font color='red'>Return Date is empty<font>

In [15]:
def fetchDate(string):
    '''
    This function tries to fetch a date (day, month and year) from a string via datetime library. 
    
    If it fails, it'll try to identify these elements from a string in the format YYYY-mm-dd.
    '''
    string = str(string)
    if string.lower() == 'nan':
        return (np.NAN, np.NAN)
    
    try:
        date = datetime.datetime.strptime(string, '%Y-%m-%d')
        return (date.month, date.year)
    
    except:
        str_list = string.split('-')
        year = str_list[0]
        month = str_list[1]
        
        return (month, year)
        

In [16]:
poliqueta['determined_month'] = poliqueta['Determined date 1'].apply(lambda x: fetchDate(x)[0])
poliqueta['start_month'] = poliqueta['Start Date'].apply(lambda x: fetchDate(x)[0])
poliqueta['cataloged_month'] = poliqueta['Cataloged Date'].apply(lambda x: fetchDate(x)[0])

poliqueta['determined_year'] = poliqueta['Determined date 1'].apply(lambda x: fetchDate(x)[1])
poliqueta['start_year'] = poliqueta['Start Date'].apply(lambda x: fetchDate(x)[1])
poliqueta['cataloged_year'] = poliqueta['Cataloged Date'].apply(lambda x: fetchDate(x)[1])

converting to int

In [17]:
def str_with_nan2int(string):
    try:
        if not np.isnan(string):
            return int(string)
        else:
            return np.NAN
    except:
        if str(string).lower() == 'nan':
            return np.NaN
        else:
            return int(str(string))

In [18]:
poliqueta['determined_month'] = poliqueta['determined_month'].apply(str_with_nan2int) #has NaN
poliqueta['start_month'] = poliqueta['start_month'].apply(str_with_nan2int) #has NaN
poliqueta['cataloged_month'] = poliqueta['cataloged_month'].apply(str_with_nan2int) #has NaN

poliqueta['determined_year'] = poliqueta['determined_year'].apply(str_with_nan2int) #has NaN
poliqueta['start_year'] = poliqueta['start_year'].apply(str_with_nan2int) #has NaN
poliqueta['cataloged_year'] = poliqueta['cataloged_year'].apply(str_with_nan2int) #has NaN

<br>

## Adjusting `Depth` columns

<font color='red'>**p.s.:** I'm assuming it's all on the same measure unit (in meters) </font>

In [19]:
poliqueta['min_depth'] = poliqueta['Min Depth'].astype(float)
poliqueta['max_depth'] = poliqueta['Max Depth'].astype(float)

<br>

## Adjusting Latitude and Longitude

In [20]:
poliqueta['Latitude'] = poliqueta['Latitude'].apply(convert2float)
poliqueta['Longitude'] = poliqueta['Longitude'].apply(convert2float)

<br>

## Adjusting locality columns

Continent
WaterBody
Country
State
County
Locality Name

In [21]:
# colunas estão boas. Inserir tratamento depois, se necessário

<br>

## Adjusting Types

In [22]:
# inserir depois (Ainda não foi necessário)
# poliqueta['Type Status 1'].str.strip().str.lower().str.capitalize()

<br>

## Adjusting `Order` column

In [23]:
def correct_order(order):
    order = str(order)
    
    if order.lower() == 'nan':
        return np.NAN
    else:
        return order.strip().capitalize()

In [24]:
poliqueta['Order 1'] = poliqueta['Order 1'].apply(correct_order)

<br>

## Selecting Subset of Main DB

In [25]:
# 'genero_e_especie_ent', 'genero_e_especie_atual'
selected_columns = ['Catalog Number','Cataloged Date','Determined date 1','Start Date',
                    'start_year', 'start_month', 'determined_year', 'determined_month',
                    'cataloged_month', 'cataloged_year',
                    'Class 1','Kingdom', 
                    'Genus 1', 'Species 1', 'Family 1', 'Phylum 1','Order 1','Type Status 1',
                    'Species Author 1', 'Species Author Year 1','Determiner Last Name 1',
                    'CollectorFirst_and_LastName', 'CollectorFirst_and_LastName2',
                    'Collector First Name 1', 'Collector Last Name 1',
                    'Qualifier 1', 'min_depth', 'max_depth',
                    'Latitude', 'Longitude', 'Continent',
                    'WaterBody', 'Country', 'State', 'County', 'Locality Name']

In [26]:
NewTable = poliqueta[selected_columns].copy()

## Renaming columns

Setting new standardized column names to facilitate future steps.

In [27]:
renames = {
    'Catalog Number':'catalog_number',
    'Cataloged Date':'cataloged_date',
    'Determined Date 1':'determined_date',
    'Class 1':'class',
    'Kingdom':'kingdom',
    'Genus 1':'genus',
    'Species 1':'species',
    'Family 1':'family',
    'Phylum 1':'phylum',
    'Subespecie_atual':'subespecie_atual',
    'Type Status 1':'type',
    'Order 1':'order',
    'Species Author 1':'author',
    'Species Author Year 1':'author_year',
    'Determiner Last Name 1':'determiner_full_name',
    'Collector First Name 1':'collector_first_name',
    'Collector Last Name 1':'collector_last_name',
    'Qualifier 1':'qualifier',
    'Latitude':'lat',
    'Longitude':'long', 
    'Continent':'continent',
    'WaterBody':'water_body',
    'Country':'country',
    'State':'state',
    'County':'county',
    'Locality Name':'locality',
    'CollectorFirst_and_LastName':'collector_full_name',
    'CollectorFirst_and_LastName2':'collector_full_name2'
}

In [28]:
NewTable = NewTable.rename(columns=renames)

<br>

## Exporting to `CSV`

name: <font color='blue'>./src/treated_db.csv</font>
sep: ';'
encoding: 'utf-8-sig'

In [29]:
NewTable.to_csv('./data/treated_db.csv', sep=';', encoding='utf-8')

-----

<br>

# MNRJP27.07.2020 - visualização.xls

## Importing data

In [30]:
excel = pd.ExcelFile('./data/MNRJP27.07.2020 - visualização.xls')
sheet_name = excel.sheet_names

print('The excel file contains the following sheets:', sheet_name)
print('\nDatabase is in sheet:', sheet_name[0])

The excel file contains the following sheets: ['Sheet0', 'Planilha1', 'Conferir coordenadas']

Database is in sheet: Sheet0


In [31]:
db = excel.parse(sheet_name[0], sep=';', encoding='utf-8-sig')
poliqueta2 = db.copy()

print(f'The database has {db.shape[0]} rows and {db.shape[1]} columns.')

The database has 2692 rows and 93 columns.


## Adjusting column names

### removing '\n', '\t', and other special characters

In [32]:
poliqueta2.columns = [str(col).replace(r'\n','') for col in poliqueta2.columns]

## Adjusting Determiners and Collectors Names

<font color='red' size='5'>**p.s.:** Determiner Last Name 1 contains the FULL name of the Researcher </font>

Determiner Last Name 2 is all empty.

In [33]:
names_col = ['Collector Last Name 1', 'Collector First Name 1', 
             'Collector Last Name 2', 'Collector First Name 2', 'Collector Last Name 3', 
             'Collector First Name 3', 'Collector Last Name 4', 'Collector First Name 4']

In [34]:
for name_col in names_col:
    if 'last' in name_col.lower():
        poliqueta2[name_col] = poliqueta2[name_col].apply(lambda x: treat_names(x, pos='full'))
    else:
        poliqueta2[name_col] = poliqueta2[name_col].apply(treat_names)

In [35]:
poliqueta2['Determiner Last Name1'] = poliqueta2['Determiner Last Name1'].str.strip().str.lower().str.capitalize()
poliqueta2['Determiner First Name1'] = poliqueta2['Determiner First Name1'].str.strip().str.lower().str.capitalize()

### creating column joining First and Last names

I'm doing this only for the first collectors and determiners

In [36]:
poliqueta2['DeterminerFirst_and_LastName'] = poliqueta2['Determiner First Name1'] + ' ' + poliqueta2['Determiner Last Name1']

poliqueta2['CollectorFirst_and_LastName'] = poliqueta2['Collector First Name 1'] + ' ' + poliqueta2['Collector Last Name 1']
poliqueta2['CollectorFirst_and_LastName2'] = poliqueta2['Collector First Name 2'] + ' ' + poliqueta2['Collector Last Name 2']

## Treating taxon columns

In [37]:
taxon_columns = ['Kingdom', 'Phylum 1', 'Class 1', 'Order 1', 'Family 1', 'Genus 1',
                 'Species 1']  # selecting taxonomy columns

treat_taxon_columns(poliqueta2, taxon_columns)

Adjusting column Kingdom
Adjusting column Phylum 1
Adjusting column Class 1
Adjusting column Order 1
Adjusting column Family 1
Adjusting column Genus 1
Adjusting column Species 1


## Adjusting Genus and Species

In [38]:
# dica da Manoela: epiteto especifico deve ser todo minusculo (especie e subespecie, nesse caso)
poliqueta2['Species 1'] = poliqueta2['Species 1'].str.lower()

# gênero é escrito com primeira letra maiúscula
poliqueta2['Genus 1'] = poliqueta2['Genus 1'].str.lower().str.capitalize()

<br>

## adding `Genero` and `Especie`together (they completely identify each animal's species)

In [39]:
poliqueta2['genus_and_species'] = poliqueta2['Genus 1'] + ' ' + poliqueta2['Species 1']

poliqueta2['genus_and_species'] = poliqueta2['genus_and_species'].str.lower().str.capitalize()

## Catching Month and Year

date_columns= ['Cataloged Date','Return Date','Determined date 1', 'Start Date]

format: YYYY-mm-dd

<font color='red'>Return Date is empty<font>

In [40]:
def fetchDate(string):
    '''
    This function tries to fetch a date (day, month and year) from a string via datetime library. 
    
    If it fails, it'll try to identify these elements from a string in the format YYYY-mm-dd.
    '''
    string = str(string)
    if string.lower() == 'nan':
        return (np.NAN, np.NAN)
    
    try:
        date = datetime.datetime.strptime(string, '%Y-%m-%d')
        return (date.month, date.year)
    
    except:
        str_list = string.split('-')
        year = str_list[0]
        month = str_list[1]
        
        return (month, year)
        

In [41]:
# poliqueta2['determined_month'] = poliqueta2['Determined date 1'].apply(lambda x: fetchDate(x)[0])
poliqueta2['start_month'] = poliqueta2['Start Date'].apply(lambda x: fetchDate(x)[0])
poliqueta2['cataloged_month'] = poliqueta2['Cataloged Date'].apply(lambda x: fetchDate(x)[0])

# poliqueta['determined_year'] = poliqueta['Determined date 1'].apply(lambda x: fetchDate(x)[1])
poliqueta2['start_year'] = poliqueta2['Start Date'].apply(lambda x: fetchDate(x)[1])
poliqueta2['cataloged_year'] = poliqueta2['Cataloged Date'].apply(lambda x: fetchDate(x)[1])

converting to int

In [42]:
def str_with_nan2int(string):
    try:
        if not np.isnan(string):
            return int(string)
        else:
            return np.NAN
    except:
        if str(string).lower() == 'nan':
            return np.NaN
        else:
            return int(str(string))

In [43]:
# poliqueta2['determined_month'] = poliqueta2['determined_month'].apply(str_with_nan2int) #has NaN
poliqueta2['start_month'] = poliqueta2['start_month'].apply(str_with_nan2int) #has NaN
poliqueta2['cataloged_month'] = poliqueta2['cataloged_month'].apply(str_with_nan2int) #has NaN

# poliqueta2['determined_year'] = poliqueta2['determined_year'].apply(str_with_nan2int) #has NaN
poliqueta2['start_year'] = poliqueta2['start_year'].apply(str_with_nan2int) #has NaN
poliqueta2['cataloged_year'] = poliqueta2['cataloged_year'].apply(str_with_nan2int) #has NaN

<br>

## Adjusting `Depth` columns

<font color='red'>**p.s.:** I'm assuming it's all on the same measure unit (in meters) </font>

In [44]:
poliqueta2['min_depth'] = poliqueta2['Min Depth'].astype(float)
poliqueta2['max_depth'] = poliqueta2['Max Depth'].astype(float)

<br>

## Adjusting Latitude and Longitude

In [45]:
poliqueta2['Latitude'] = poliqueta2['Latitude1'].apply(convert2float)
poliqueta2['Longitude'] = poliqueta2['Longitude1'].apply(convert2float)

<br>

## Adjusting locality columns

Continent
WaterBody
Country
State
County
Locality Name

In [46]:
for col in poliqueta2.columns:
    if 'depth' in col.lower():
        print(col)

Min Depth
Max Depth
min_depth
max_depth


### treating continent

In [47]:
def correct_continent(string):
    string = str(string)

    continent_correction = {
        'America Central':'Central America',
        'Asia Oriental':'Asia',  # there's just one register of Asia Oriental
        'European':'Europe',
    }
    
    if string in continent_correction.keys():
        return continent_correction[string]  # returns corrected continent
    else:
        return string

In [48]:
poliqueta2['Continent'] = poliqueta2['Continent'].str.strip().apply(correct_continent).value_counts()

### treating State column

In [49]:
def remove_accents(string):
    string = str(string).strip()
    
    return unidecode.unidecode(string)

In [50]:
poliqueta2['State'] = poliqueta2['State'].str.strip().apply(remove_accents).value_counts()

In [51]:
# other columns will remain the same, while further treatment doesn't seem necessary

<br>

## Adjusting Types

In [52]:
# inserir depois (Ainda não foi necessário)
# poliqueta['Type Status 1'].str.strip().str.lower().str.capitalize()

<br>

## Adjusting `Order` column

In [53]:
def correct_order(order):
    order = str(order)
    
    if order.lower() == 'nan':
        return np.NAN
    else:
        return order.strip().capitalize()

In [54]:
poliqueta2['Order 1'] = poliqueta2['Order 1'].apply(correct_order)

<br>

## Selecting Subset of Main DB

In [55]:
# Não tem Determined Date
selected_columns = ['Catalog Number','Cataloged Date','Start Date',
                    'start_year', 'start_month',
                    'cataloged_month', 'cataloged_year',
                    'Class 1','Kingdom', 
                    'Genus 1', 'Species 1', 'Family 1', 'Phylum 1','Order 1','Type Status 1',
                    'Species Author 1', 'Species Author Year 1','Determiner Last Name1','Determiner First Name1',
                    'DeterminerFirst_and_LastName','CollectorFirst_and_LastName','CollectorFirst_and_LastName2', 
                    'Collector First Name 1', 'Collector Last Name 1',
                    'Qualifier 1', 'min_depth', 'max_depth',
                    'Latitude', 'Longitude', 'Continent',
                    'WaterBody', 'Country', 'State', 'County', 'Locality Name']

In [56]:
NewTable2 = poliqueta2[selected_columns].copy()

## Renaming columns

Setting new standardized column names to facilitate future steps.

In [57]:
renames = {
    'Catalog Number':'catalog_number',
    'Cataloged Date':'cataloged_date',
    'Determined Date 1':'determined_date',
    'Class 1':'class',
    'Kingdom':'kingdom',
    'Genus 1':'genus',
    'Species 1':'species',
    'Family 1':'family',
    'Phylum 1':'phylum',
    'Subespecie_atual':'subespecie_atual',
    'Type Status 1':'type',
    'Order 1':'order',
    'Species Author 1':'author',
    'Species Author Year 1':'author_year',
    'Determiner Last Name1':'determiner_last_name',
    'Determiner First Name1':'determiner_first_name',
    'Collector First Name 1':'collector_first_name',
    'Collector Last Name 1':'collector_last_name',
    'Qualifier 1':'qualifier',
    'Latitude':'lat',
    'Longitude':'long', 
    'Continent':'continent',
    'WaterBody':'water_body',
    'Country':'country',
    'State':'state',
    'County':'county',
    'Locality Name':'locality',
    'CollectorFirst_and_LastName':'collector_full_name',
    'CollectorFirst_and_LastName2':'collector_full_name2',
    'DeterminerFirst_and_LastName':'determiner_full_name'
}

In [58]:
NewTable2 = NewTable2.rename(columns=renames)

<br>

## Exporting to `CSV`

name: <font color='blue'>./src/treated_db.csv</font>
sep: ';'
encoding: 'utf-8-sig'

In [59]:
NewTable2.to_csv('./data/treated_db2.csv', sep=';', encoding='utf-8')

-----

<br>

# Merging both databases

p.s.: this will represent the whole 'poliqueta' collection 

In [60]:
# merge NewTable and NewTable2
set(NewTable.columns).difference(set(NewTable2.columns))

{'Determined date 1', 'determined_month', 'determined_year'}

In [61]:
merged_table = pd.concat([NewTable, NewTable2], join='outer')

In [62]:
def create_prefix(catalog):
    catalog = str(catalog)
    
    if 'IBUFRJ' in catalog.upper():
        return 'IBUFRJ'
    elif 'MNRJP' in catalog.upper():
        return 'MNRJP'
    else:
        return None

In [63]:
# creating column collection_prefix IBUFRJ and MNRJP (to identify origin of that info.)
merged_table['collection_prefix'] = merged_table['catalog_number'].apply(create_prefix)

In [64]:
merged_table.to_csv('./data/merged_db.csv', sep=';', encoding='utf-8')